# Домашнее задание № 11. Машинный перевод

## Задание 1 (6 баллов + 2 доп балла).
Нужно обучить трансформер на этом же или на другом корпусе (можно взять другую языковую пару с того же сайте) и оценивать его на всей тестовой выборке (а не на 10 примерах как сделал я). 

Чтобы получить 2 доп балла вам нужно будет придумать как оптимизировать функцию translate. Подсказка: модель может предсказывать батчами.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# в русскоязычных данных есть \xa0 вместо пробелов, он может некорректно обрабатываться токенизатором
text = open('opus.en-ru-train.ru.txt', encoding="UTF8").read().replace('\xa0', ' ')
f = open('opus.en-ru-train.ru.txt', 'w', encoding="UTF8")
f.write(text)
f.close()

In [6]:
text = open('opus.en-ru-test.ru.txt', encoding="UTF8").read().replace('\xa0', ' ')
f = open('opus.en-ru-test.ru.txt', 'w', encoding="UTF8")
f.write(text)
f.close()

In [46]:
en_sents = open('opus.en-ru-train.en.txt', encoding="UTF8").read().splitlines()
ru_sents = open('opus.en-ru-train.ru.txt', encoding="UTF8").read().splitlines()

In [47]:
en_sents[-1], ru_sents[-1]

('So what are you thinking?', 'Ну и что ты думаешь?')

In [48]:
tokenizer_en = Tokenizer(BPE())
tokenizer_en.pre_tokenizer = Whitespace()
trainer_en = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer_en.train(files=["opus.en-ru-train.en.txt"], trainer=trainer_en)

tokenizer_ru = Tokenizer(BPE())
tokenizer_ru.pre_tokenizer = Whitespace()
trainer_ru = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer_ru.train(files=["opus.en-ru-train.ru.txt"], trainer=trainer_ru)

In [49]:
# раскоментируйте эту ячейку при обучении токенизатора
# а потом снова закоментируйте чтобы при перезапуске не перезаписать токенизаторы
# tokenizer_en.save('tokenizer_en')
# tokenizer_ru.save('tokenizer_ru')

In [50]:
tokenizer_en = Tokenizer.from_file("tokenizer_en")
tokenizer_ru = Tokenizer.from_file("tokenizer_ru")

In [13]:
def encode(text, tokenizer, max_len):
    return [tokenizer.token_to_id('[CLS]')] + tokenizer.encode(text).ids[:max_len] + [tokenizer.token_to_id('[SEP]')]

In [51]:
# важно следить чтобы индекс паддинга совпадал в токенизаторе с value в pad_sequences
PAD_IDX = tokenizer_ru.token_to_id('[PAD]')
PAD_IDX

3

In [26]:
# ограничимся длинной в 30 и 35 (разные чтобы показать что в seq2seq не нужна одинаковая длина)
max_len_en, max_len_ru = 30, 35

In [52]:
X_en = [encode(t, tokenizer_en, max_len_en) for t in en_sents]
X_ru = [encode(t, tokenizer_ru, max_len_ru) for t in ru_sents]

In [53]:
# миллион примеров
X_en.__len__(), X_ru.__len__()

(1000000, 1000000)

In [34]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, texts_en, texts_ru):
        self.texts_en = [torch.LongTensor(sent) for sent in texts_en]
        self.texts_en = torch.nn.utils.rnn.pad_sequence(self.texts_en, padding_value=PAD_IDX)

        self.texts_ru = [torch.LongTensor(sent) for sent in texts_ru]
        self.texts_ru = torch.nn.utils.rnn.pad_sequence(self.texts_ru, padding_value=PAD_IDX)

        self.length = len(texts_en)

    def __len__(self):
        return self.length

    def __getitem__(self, index):

        ids_en = self.texts_en[:, index]
        ids_ru = self.texts_ru[:, index]

        return ids_en, ids_ru

In [54]:
X_en_train, X_en_valid, X_ru_train, X_ru_valid = train_test_split(X_en, X_ru, test_size=0.05)

In [55]:
training_set = Dataset(X_en_train, X_ru_train)
training_generator = torch.utils.data.DataLoader(training_set, batch_size=200, shuffle=True, )

In [56]:
valid_set = Dataset(X_en_valid, X_ru_valid)
valid_generator = torch.utils.data.DataLoader(valid_set, batch_size=200, shuffle=True)

In [57]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 150):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
#         print('pos inp')
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
#         print('pos dec')
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
#         print('pos out')
        x = self.generator(outs)
#         print('gen')
        return x

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)
# During training, we need a subsequent word mask that will prevent model to look into the future words when making predictions. We will also need masks to hide source and target padding tokens. Below, let’s define a function that will take care of both.

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)

    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [38]:
from time import time
def train(model, iterator, optimizer, criterion, print_every=500):

    epoch_loss = []
    ac = []

    model.train()

    for i, (texts_en, texts_ru) in enumerate(iterator):
        texts_en = texts_en.T.to(DEVICE) # чтобы батч был в конце
        texts_ru = texts_ru.T.to(DEVICE) # чтобы батч был в конце

        # помимо текста в модель еще нужно передать целевую последовательность
        # но не полную а без 1 последнего элемента
        # а на выходе ожидаем, что модель сгенерирует этот недостающий элемент
        texts_ru_input = texts_ru[:-1, :]


        # в трансформерах нет циклов как в лстм
        # каждый элемент связан с каждым через аттеншен
        # чтобы имитировать последовательную обработку
        # и чтобы не считать аттеншн с паддингом
        # в трансформерах нужно считать много масок
        # подробнее про это по ссылкам выше
        (texts_en_mask, texts_ru_mask,
        texts_en_padding_mask, texts_ru_padding_mask) = create_mask(texts_en, texts_ru_input)
        logits = model(texts_en, texts_ru_input, texts_en_mask, texts_ru_mask,
                       texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
        optimizer.zero_grad()

        # сравниваем выход из модели с целевой последовательностью уже с этим последним элементом
        texts_ru_out = texts_ru[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), texts_ru_out.reshape(-1))
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())

        if not (i+1) % print_every:
            print(f'Loss: {np.mean(epoch_loss)};')

    return np.mean(epoch_loss)


def evaluate(model, iterator, criterion):

    epoch_loss = []
    epoch_f1 = []

    model.eval()
    with torch.no_grad():
        for i, (texts_en, texts_ru) in enumerate(iterator):
            texts_en = texts_en.T.to(DEVICE)
            texts_ru = texts_ru.T.to(DEVICE)

            texts_ru_input = texts_ru[:-1, :]

            (texts_en_mask, texts_ru_mask,
            texts_en_padding_mask, texts_ru_padding_mask) = create_mask(texts_en, texts_ru_input)

            logits = model(texts_en, texts_ru_input, texts_en_mask, texts_ru_mask,
                           texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)


            texts_ru_out = texts_ru[1:, :]
            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), texts_ru_out.reshape(-1))
            epoch_loss.append(loss.item())

    return np.mean(epoch_loss)

In [61]:
torch.manual_seed(0)

EN_VOCAB_SIZE = tokenizer_en.get_vocab_size()
RU_VOCAB_SIZE = tokenizer_ru.get_vocab_size()

EMB_SIZE = 256
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 2

try:
    transformer = torch.load('model')
except:
    transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                     NHEAD, EN_VOCAB_SIZE, RU_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [62]:
torch.cuda.empty_cache()

In [63]:
from timeit import default_timer as timer
NUM_EPOCHS = 100

losses = []

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train(transformer, training_generator, optimizer, loss_fn)
    end_time = timer()
    val_loss = evaluate(transformer, valid_generator, loss_fn)

    if not losses:
        print(f'First epoch - {val_loss}, saving model..')
        torch.save(transformer, 'model')

    elif val_loss < min(losses):
        print(f'Improved from {min(losses)} to {val_loss}, saving model..')
        torch.save(transformer, 'model')

    losses.append(val_loss)

    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, \
           "f"Epoch time={(end_time-start_time):.3f}s"))

C:\Users\robst\anaconda3\envs\gpt2\lib\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Loss: 7.060866549491882;
Loss: 6.5951112375259395;
Loss: 6.300608988444011;
Loss: 6.078218590974807;
Loss: 5.902960467147827;
Loss: 5.756781321684519;
Loss: 5.628788885797773;
Loss: 5.517271479129791;
Loss: 5.416293688774109;
First epoch - 4.331290998458862, saving model..
Epoch: 1, Train loss: 5.370, Val loss: 4.331,            Epoch time=513.710s
Loss: 4.390142631530762;
Loss: 4.3579894037246705;
Loss: 4.316447576522827;
Loss: 4.279523859739304;
Loss: 4.244511232757568;
Loss: 4.2106361645857495;
Loss: 4.177314560413361;
Loss: 4.146812884509563;
Loss: 4.117848366737365;
Improved from 4.331290998458862 to 3.6575626134872437, saving model..
Epoch: 2, Train loss: 4.103, Val loss: 3.658,            Epoch time=509.906s
Loss: 3.721607120513916;
Loss: 3.7105232753753663;
Loss: 3.6969984742800395;
Loss: 3.684307337641716;
Loss: 3.6689805006980896;
Loss: 3.6547887053489685;
Loss: 3.6404446503094263;
Loss: 3.6264306665062906;
Loss: 3.613754880110423;
Improved from 3.6575626134872437 to 3.322567

KeyboardInterrupt: 

In [64]:
def translate(text):


    input_ids = [tokenizer_en.token_to_id('[CLS]')] + tokenizer_en.encode(text).ids[:max_len_en] + [tokenizer_en.token_to_id('[SEP]')]
    output_ids = [tokenizer_ru.token_to_id('[CLS]')]

    input_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(input_ids)]).to(DEVICE)
    output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)]).to(DEVICE)

    (texts_en_mask, texts_ru_mask,
    texts_en_padding_mask, texts_ru_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
    logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_ru_mask,
                   texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
    pred = logits.argmax(2).item()

    while pred not in [tokenizer_ru.token_to_id('[SEP]'), tokenizer_ru.token_to_id('[PAD]')]:
        output_ids.append(pred)
        output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)]).to(DEVICE)

        (texts_en_mask, texts_ru_mask,
        texts_en_padding_mask, texts_ru_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
        logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_ru_mask,
                       texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
        pred = logits.argmax(2)[-1].item()

    return (' '.join([tokenizer_ru.id_to_token(i).replace('##', '') for i in output_ids[1:]]))

In [65]:
translate('Can you translate that?')

'Ты можешь это перевести ?'

In [66]:
import nltk

hypothesis = ['It', 'is', 'a', 'cat', 'at', 'room']
reference = ['It', 'is', 'a', 'cat', 'inside', 'the', 'room']


BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, auto_reweigh=True)
print(BLEUscore)

0.4548019047027907


In [67]:
en_test_sents = open('opus.en-ru-test.en.txt', encoding="UTF8").read().lower().splitlines()
ru_test_sents = open('opus.en-ru-test.ru.txt', encoding="UTF8").read().lower().splitlines()

In [68]:
translations = []

for i in range(len(en_test_sents)):
    translations.append(translate(en_test_sents[i]))

In [ ]:
bleus = []

for i, t in enumerate(translations):
    reference = tokenizer_ru.encode(t).tokens
    hypothesis = tokenizer_ru.encode(ru_test_sents[i]).tokens

    bleus.append(nltk.translate.bleu_score.sentence_bleu([reference], hypothesis,  auto_reweigh=True))

In [70]:
(sum(bleus)/len(bleus))*100

6.910610048631724


## Задание 2 (2 балла).
Прочитайте главу про машинный перевод у Журафски и Маннига - https://web.stanford.edu/~jurafsky/slp3/10.pdf 
Ответьте своими словами в чем заключается техника back translation? Для чего она применяется и что позволяет получить? Опишите по шагам как его применить к паре en-ru на данных из семинара.

Backtranslation - один из методов, относящихся к технике Data augmentation (увеличения данных), которая применяется при недостатке обучающих данных (часто при работе с малоресурсными языками). Идея обратного перевода опирается на предположение, что существующие небольшие параллельные корпуса можно дополнить синтетическими битекстами, основанными на материалах одноязычного корпуса. Предположим, у нас есть небольшой параллельный корпус en-ru, где en - исходный язык, ru - целевой, а также много данных на ru. При обратном переводе модель сначала обучается на существующем параллельном корпусе в обратную сторону - перевод «ЦЯ-ИЯ». Эту модель используется для перевода одноязычных данных ru на en. Таким образом мы получаем синтетический параллельный корпус, данные которого можем перемешать с изначальным параллельным корпусом и затем обучить модель на всех имеющихся данных уже в направлении «ИЯ-ЦЯ».